Klasyfikacja szkła

1. eksploracja danych – przedstaw podstawowe dane statystyczne i uwagi dotyczące cech i etykiet
zbioru danych. (10 punktów)
2. przygotowanie danych – podziel dane na zestaw uczący i walidacyjny (alternatywnie użyj wali-
dacji krzyżowej), zbadaj wpływ różnego typu przetworzenia danych na wyniki klasyfikacji (pro-
ponowane: normalizacja, standaryzacja, dyskretyzacja, selekcja cech, PCA) - czyli wykonaj po-
równanie wyników bez przetworzenia danych z rezultatami po ich przetworzeniu, wykorzystując
co najmniej 2 metody różnego typu (osobno). (30 punktów)
Bonus – usuń 5% wartości cech i przygotuj dane stosując metody radzenia sobie z brakującymi
danymi. (5 punktów)
3. klasyfikacja – przetestuj klasyfikatory i zbadaj wpływ na wyniki: naiwny klasyfikator Bayesa oraz
drzewo decyzjne używając przynajmniej 3 różnych zestawów hiperparametrów. (40 punktów)
Bonus – Przetestuj (ze zrozumieniem!) bardziej zaawansowane algorytmy, takie jak Las losowy
czy Klasyfikator wektorów nośnych (SVM, z ang. Support Vector Machines). (5 punktów)
4. ocena klasyfikacji – do porównania wyników różnego typu przygotowania danych oraz wyko-
rzystanego klasyfikatora użyj poznanych metryk oceny klasyfikacji i zinterpretuj wyniki. (20
punktów)

In [12]:
import pandas as pd

data = pd.read_csv('glass.data', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [13]:
data.shape

(214, 11)

Before browsing data statistics, we add headers to understand the data

In [14]:
header_names = ['Id','RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','glass_type']
data.columns = header_names
data.head()

,Id,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,glass_type
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


... and remove Id because we do not need to calculate median Id etc.

In [15]:
data = data.drop('Id',axis=1)
data.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,glass_type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RI          214 non-null    float64
 1   Na          214 non-null    float64
 2   Mg          214 non-null    float64
 3   Al          214 non-null    float64
 4   Si          214 non-null    float64
 5   K           214 non-null    float64
 6   Ca          214 non-null    float64
 7   Ba          214 non-null    float64
 8   Fe          214 non-null    float64
 9   glass_type  214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [17]:
data.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,glass_type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516522,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [18]:
features = ['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe']
label = ['glass_type']

X = data[features]

y = data[label]

In [19]:
X.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0


In [20]:
y.tail()

,glass_type
209,7
210,7
211,7
212,7
213,7


In [21]:
import plotly.graph_objects as go
import numpy as np

correlation = X.corr()
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = correlation.columns,
        y = correlation.index,
        z = np.array(correlation),
        text=correlation.values,
        texttemplate='%{text:.2f}'
    )
)

fig.show()

Correlation is checked, as Naie Bayes is based on the assumption that predictors are independent of each other.
There is a correlation between Refractive index and Calcium.

Define a function to avoid having code repetition in all different preprocessing methods.

In [25]:
from collections import namedtuple

ModelMetrics = namedtuple("ModelMetrics", ["matrix", "accuracy", "recall", "precision", "f1"])

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

TEST_SET_PROPORTION = 0.25

def decision_tree_process(X_param, y_param, criterion):
    X_train, X_test, y_train, y_test = train_test_split(X_param, y_param, test_size=0.25, random_state=0, stratify=y)

    # flatten
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()
    
    tree = DecisionTreeClassifier(random_state=0, criterion=criterion)
    tree.fit(X_train, y_train)
    y_pred = tree.predict(X_test)

    metrics = ModelMetrics(confusion_matrix(y_test, y_pred),
                           accuracy_score(y_test, y_pred),
                           recall_score(y_test, y_pred, average="weighted"),
                           precision_score(y_test, y_pred, average="weighted"),
                           f1_score(y_test, y_pred, average="weighted")
                           )
    
    return metrics


Gaussian Naive Bayes – This is a variant of Naive Bayes which supports continuous values and has an assumption that each class is normally distributed. 

In [50]:
from sklearn.naive_bayes import GaussianNB

TEST_SET_PROPORTION = 0.25

def bayes_process(X_param, y_param):
    X_train, X_test, y_train, y_test = train_test_split(X_param, y_param, test_size=0.25, random_state=0, stratify=y)

    # flatten
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()
    
    bayes = GaussianNB()
    bayes.fit(X_train, y_train)
    y_pred = bayes.predict(X_test)

    metrics = ModelMetrics(confusion_matrix(y_test, y_pred),
                           accuracy_score(y_test, y_pred),
                           recall_score(y_test, y_pred, average="weighted"),
                           precision_score(y_test, y_pred, average="weighted"),
                           f1_score(y_test, y_pred, average="weighted")
                           )
    
    return metrics


Gini Impurity is a measurement used to build Decision Trees to determine how the features of a dataset should split nodes to form the tree. More precisely, the Gini Impurity of a dataset is a number between 0-0.5, which indicates the likelihood of new, random data being misclassified if it were given a random class label according to the class distribution in the dataset.

In [77]:
unprocessed_gini_tree_metrics = decision_tree_process(X, y, "gini")
unprocessed_gini_tree_metrics

ModelMetrics(matrix=array([[13,  1,  3,  0,  0,  1],
       [ 3, 14,  0,  2,  0,  0],
       [ 1,  2,  1,  0,  0,  0],
       [ 0,  1,  0,  1,  0,  1],
       [ 0,  1,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  8]], dtype=int64), accuracy=0.7037037037037037, recall=0.7037037037037037, precision=0.7067538126361655, f1=0.7002939447383892)

In [84]:
unprocessed_entropy_tree_metrics = decision_tree_process(X, y, "entropy")
unprocessed_entropy_tree_metrics

c:\Users\szymo\source\repos\jnotebooks\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



ModelMetrics(matrix=array([[ 8,  5,  5,  0,  0,  0],
       [ 3, 14,  0,  2,  0,  0],
       [ 2,  2,  0,  0,  0,  0],
       [ 1,  0,  0,  2,  0,  0],
       [ 0,  0,  0,  0,  0,  2],
       [ 2,  1,  0,  0,  0,  5]], dtype=int64), accuracy=0.5370370370370371, recall=0.5370370370370371, precision=0.5241702741702742, f1=0.5276632785001986)

In [78]:
from sklearn.preprocessing import Normalizer
X_normalizer = pd.DataFrame(Normalizer().fit_transform(X), columns=X.columns)

normalizer_gini_tree_metrics = decision_tree_process(X_normalizer, y, "gini")
normalizer_gini_tree_metrics

ModelMetrics(matrix=array([[ 8,  2,  7,  0,  0,  1],
       [ 3, 14,  1,  1,  0,  0],
       [ 3,  0,  1,  0,  0,  0],
       [ 0,  1,  0,  1,  0,  1],
       [ 0,  0,  0,  0,  2,  0],
       [ 0,  0,  0,  0,  0,  8]], dtype=int64), accuracy=0.6296296296296297, recall=0.6296296296296297, precision=0.6718003250682989, f1=0.6426717315606205)

In [85]:
normalizer_entropy_tree_metrics = decision_tree_process(X_normalizer, y, "entropy")
normalizer_entropy_tree_metrics

ModelMetrics(matrix=array([[10,  2,  6,  0,  0,  0],
       [ 3, 13,  2,  1,  0,  0],
       [ 3,  0,  1,  0,  0,  0],
       [ 0,  1,  1,  1,  0,  0],
       [ 0,  0,  0,  0,  2,  0],
       [ 0,  1,  1,  0,  0,  6]], dtype=int64), accuracy=0.6111111111111112, recall=0.6111111111111112, precision=0.6970934838581897, f1=0.6463135871632603)

In [79]:
from sklearn.preprocessing import StandardScaler

X_scaler = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)

scaler_gini_tree_metrics = decision_tree_process(X_scaler, y, "gini")
scaler_gini_tree_metrics

ModelMetrics(matrix=array([[13,  1,  3,  0,  0,  1],
       [ 3, 14,  0,  2,  0,  0],
       [ 1,  2,  1,  0,  0,  0],
       [ 0,  1,  0,  1,  0,  1],
       [ 0,  1,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  8]], dtype=int64), accuracy=0.7037037037037037, recall=0.7037037037037037, precision=0.7067538126361655, f1=0.7002939447383892)

In [86]:
scaler_entropy_tree_metrics = decision_tree_process(X_scaler, y, "entropy")
scaler_entropy_tree_metrics

c:\Users\szymo\source\repos\jnotebooks\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



ModelMetrics(matrix=array([[ 8,  5,  5,  0,  0,  0],
       [ 3, 14,  0,  2,  0,  0],
       [ 2,  2,  0,  0,  0,  0],
       [ 1,  0,  0,  2,  0,  0],
       [ 0,  0,  0,  0,  0,  2],
       [ 2,  1,  0,  0,  0,  5]], dtype=int64), accuracy=0.5370370370370371, recall=0.5370370370370371, precision=0.5241702741702742, f1=0.5276632785001986)

In [80]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

estimator = SVR(kernel="linear")
X_fs = pd.DataFrame(RFE(estimator, n_features_to_select=3).fit_transform(X, y.values.ravel()))

fs_gini_tree_metrics = decision_tree_process(X_fs, y, "gini")
fs_gini_tree_metrics

ModelMetrics(matrix=array([[13,  0,  5,  0,  0,  0],
       [ 1, 13,  3,  2,  0,  0],
       [ 2,  1,  1,  0,  0,  0],
       [ 0,  0,  1,  2,  0,  0],
       [ 0,  0,  0,  0,  0,  2],
       [ 0,  0,  2,  0,  1,  5]], dtype=int64), accuracy=0.6296296296296297, recall=0.6296296296296297, precision=0.7373236331569665, f1=0.6718892944383141)

In [87]:
fs_entropy_tree_metrics = decision_tree_process(X_fs, y, "entropy")
fs_entropy_tree_metrics

ModelMetrics(matrix=array([[12,  0,  6,  0,  0,  0],
       [ 3, 12,  2,  2,  0,  0],
       [ 2,  1,  1,  0,  0,  0],
       [ 0,  0,  1,  2,  0,  0],
       [ 0,  0,  0,  0,  0,  2],
       [ 0,  0,  2,  0,  1,  5]], dtype=int64), accuracy=0.5925925925925926, recall=0.5925925925925926, precision=0.6998511655374401, f1=0.632231040564374)

In [52]:
unprocessed_bayes_metrics = bayes_process(X, y)
unprocessed_bayes_metrics

ModelMetrics(matrix=array([[11,  2,  4,  0,  1,  0],
       [14,  3,  0,  2,  0,  0],
       [ 4,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  1],
       [ 0,  0,  0,  0,  2,  0],
       [ 0,  0,  0,  0,  0,  8]], dtype=int64), accuracy=0.48148148148148145, recall=0.48148148148148145, precision=0.5217042713211296, f1=0.44480054433120775)

In [53]:
normalizer_bayes_metrics = bayes_process(X_normalizer, y)
normalizer_bayes_metrics

ModelMetrics(matrix=array([[11,  1,  5,  0,  1,  0],
       [14,  2,  0,  2,  0,  1],
       [ 3,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  1],
       [ 0,  0,  0,  0,  2,  0],
       [ 0,  0,  0,  0,  0,  8]], dtype=int64), accuracy=0.46296296296296297, recall=0.46296296296296297, precision=0.47786596119929453, f1=0.41367482043810555)

In [56]:
scaler_bayes_metrics = bayes_process(X_scaler, y)
scaler_bayes_metrics

ModelMetrics(matrix=array([[11,  2,  4,  0,  1,  0],
       [14,  3,  0,  2,  0,  0],
       [ 4,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  1],
       [ 0,  0,  0,  0,  2,  0],
       [ 0,  0,  0,  0,  0,  8]], dtype=int64), accuracy=0.48148148148148145, recall=0.48148148148148145, precision=0.5217042713211296, f1=0.44480054433120775)

In [59]:
fs_bayes_metrics = bayes_process(X_fs, y)
fs_bayes_metrics

ModelMetrics(matrix=array([[11,  0,  6,  0,  1,  0],
       [11,  4,  2,  2,  0,  0],
       [ 3,  0,  1,  0,  0,  0],
       [ 0,  1,  0,  1,  0,  1],
       [ 0,  0,  0,  0,  0,  2],
       [ 0,  2,  0,  0,  1,  5]], dtype=int64), accuracy=0.4074074074074074, recall=0.4074074074074074, precision=0.4670664315108759, f1=0.4013118664281455)

We visualize the metrics to compare different techniques


In [88]:
results = {
    'gini_tree_unprocessed': [unprocessed_gini_tree_metrics.accuracy, unprocessed_gini_tree_metrics.recall, unprocessed_gini_tree_metrics.precision, unprocessed_gini_tree_metrics.f1],
    'gini_tree_normalizer': [normalizer_gini_tree_metrics.accuracy, normalizer_gini_tree_metrics.recall, normalizer_gini_tree_metrics.precision, normalizer_gini_tree_metrics.f1],
    'gini_tree_scaler': [scaler_gini_tree_metrics.accuracy, scaler_gini_tree_metrics.recall, scaler_gini_tree_metrics.precision, scaler_gini_tree_metrics.f1],
    'gini_tree_fs': [fs_gini_tree_metrics.accuracy, fs_gini_tree_metrics.recall, fs_gini_tree_metrics.precision, fs_gini_tree_metrics.f1],
    'ent_tree_unprocessed': [unprocessed_entropy_tree_metrics.accuracy, unprocessed_entropy_tree_metrics.recall, unprocessed_entropy_tree_metrics.precision, unprocessed_entropy_tree_metrics.f1],
    'ent_tree_normalizer': [normalizer_entropy_tree_metrics.accuracy, normalizer_entropy_tree_metrics.recall, normalizer_entropy_tree_metrics.precision, normalizer_entropy_tree_metrics.f1],
    'ent_tree_scaler': [scaler_entropy_tree_metrics.accuracy, scaler_entropy_tree_metrics.recall, scaler_entropy_tree_metrics.precision, scaler_entropy_tree_metrics.f1],
    'ent_tree_fs': [fs_entropy_tree_metrics.accuracy, fs_entropy_tree_metrics.recall, fs_entropy_tree_metrics.precision, fs_entropy_tree_metrics.f1],
    'bayes_unprocessed': [unprocessed_bayes_metrics.accuracy, unprocessed_bayes_metrics.recall, unprocessed_bayes_metrics.precision, unprocessed_bayes_metrics.f1],
    'bayes_normalizer': [normalizer_bayes_metrics.accuracy, normalizer_bayes_metrics.recall, normalizer_bayes_metrics.precision, normalizer_bayes_metrics.f1],
    'bayes_scaler': [scaler_bayes_metrics.accuracy, scaler_bayes_metrics.recall, scaler_bayes_metrics.precision, scaler_bayes_metrics.f1],
    'bayes_fs': [fs_bayes_metrics.accuracy, fs_bayes_metrics.recall, fs_bayes_metrics.precision, fs_bayes_metrics.f1]
}

results_df = pd.DataFrame(results)

fig1 = go.Figure()
fig1.add_trace(
    go.Heatmap(
        x = results_df.columns,
        y = ['accuracy', 'recall', 'precision', 'f1'],
        z = np.array(results_df),
        text=results_df.values,
        texttemplate='%{text:.7f}'
    )
)
fig1.show()

Recall is always equal to accuracy in my example. Since we are looking at all the classes together, each prediction error (X is misclassified as Y) is a False Positive for Y, and a False Negative for X. Recall must be equal to accuracy, this is not a coincidence. dodać wzró

SOURCES:
- SIIIW lecture
- documentation for libraries, especially of sklearn
- https://www.analyticsvidhya.com/blog/2021/01/gaussian-naive-bayes-with-hyperpameter-tuning/
- https://bulldogjob.pl/readme/popularne-metody-selekcji-cech-w-data-science
- https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1
- https://realpython.com/pandas-python-explore-dataset/
- stats.stackexchange
- https://www.learndatasci.com/glossary/gini-impurity/

LIBRARIES USED:
- pandas - used to read, store and process tabular data
- scikit_learn - used to split training and validation set, and to run all models
- plotly - used to plot data on graphs
- numpy - used to create an array to pass to plotly as one of parameters. Normally could be used to perform more complicated tasks with arrays of numbers.
